In [130]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


In [131]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
if "Survived" not in df_test.columns:
    df_test['Survived'] = 0

In [132]:
def Preprocess(df_train, df_test):
    df = pd.concat([df_train, df_test], axis = 0)
    df = df.drop(['Name', 'Ticket'], axis = 1)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('X000')
    df['Embarked'] = df['Embarked'].fillna('X')
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

    df['cabin_letter'] = df['Cabin'].str.extract(r'([a-zA-z]+)', expand = False)
    df['cabin_number'] = df['Cabin'].str.extract(r'(\d+)', expand = False)
    df = df.drop('Cabin', axis = 1)

    df = pd.get_dummies(df, columns = ['cabin_letter'], prefix = 'cabin')
    df = pd.get_dummies(df, columns = ['Embarked'], prefix = 'Embarked')
    df = pd.get_dummies(df, columns = ['Sex'], prefix = 'Sex')

    df = df.drop('cabin_X', axis = 1)
    df = df.drop('Embarked_X', axis = 1)

    df['cabin_number'] = df['cabin_number'].fillna(0)
    df['cabin_number'] = pd.to_numeric(df['cabin_number'])

    df['Pclass_bin_Fare'] = df['Fare'] // df['Pclass']
    df['Pclass_bin_sex'] = df['Pclass'] - df['Sex_female']

    df_train = df[:len(df_train)]
    df_test = df[len(df_train):]

    df_test = df_test.drop('Survived', axis = 1)

    return df_train, df_test

In [133]:
train_df, test_df = Preprocess(df_train, df_test)

In [134]:
train_df.corr()['Survived']

PassengerId       -0.005007
Survived           1.000000
Pclass            -0.338481
Age               -0.070323
SibSp             -0.035322
Parch              0.081629
Fare               0.257307
cabin_number       0.229756
cabin_A            0.022287
cabin_B            0.175095
cabin_C            0.114652
cabin_D            0.150716
cabin_E            0.145321
cabin_F            0.057935
cabin_G            0.016040
cabin_T           -0.026456
Embarked_C         0.168240
Embarked_Q         0.003650
Embarked_S        -0.155660
Sex_female         0.543351
Sex_male          -0.543351
Pclass_bin_Fare    0.267823
Pclass_bin_sex    -0.533994
Name: Survived, dtype: float64

In [135]:
X = train_df.drop('Survived', axis = 1)
y = train_df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
y_train = np.reshape(y_train, (-1, 1))

In [136]:
X_train.shape, y_train.shape

((712, 22), (712, 1))

In [137]:
model_1 = LogisticRegression()
model_1.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [138]:
y_pred = model_1.predict(X_test)

In [139]:
accuracy_score(y_test, y_pred)

0.776536312849162

In [140]:
model_2 = XGBClassifier()
model_2.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [141]:
y_pred = model_2.predict(X_test)
accuracy_score(y_test, y_pred)

0.8044692737430168

In [142]:
model_3 = RandomForestClassifier()
model_3.fit(X_train, y_train)

/tmp/ipykernel_36/2970177101.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_3.fit(X_train, y_train)


RandomForestClassifier()

In [143]:
y_pred = model_3.predict(X_test)
accuracy_score(y_test, y_pred)

0.8212290502793296

In [145]:
pred = model_3.predict(test_df)
final = pd.DataFrame()
final['PassengerID'] = test_df['PassengerId']
final['Survived'] = pred

final.to_csv('submission-titanic.csv', index = False)